In [1]:
import random

import cv2
import numpy as np

Download images from directory.

In [2]:
from image import read_image, show_image

In [3]:
images, image_names = read_image("InputImages")

In [4]:
image = images[1]

In [5]:
show_image("OutputImage", image)

# Stratification

Get lines from the image.

In [6]:
from line import get_lines, draw_line

In [7]:
lines = get_lines(image)
len(lines)

40

Plot lines.

In [ ]:
ouput_image = image.copy()
# Getting the lines form the image
for line in lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(ouput_image, line, (r, g, b))
show_image("OutputImage", ouput_image)

Cluster lines to find vanishing points.

In [8]:
from clustering import cluster_lines

In [9]:
n_clusters = 3

In [10]:
clusters = cluster_lines(lines, n_clusters=n_clusters)
clusters

defaultdict(list,
            {0: [array([[443.        ,   0.75049156]], dtype=float32),
              array([[261.       ,   0.6981317]], dtype=float32),
              array([[245.       ,   0.6806784]], dtype=float32),
              array([[569.       ,   0.7853982]], dtype=float32),
              array([[361.      ,   0.715585]], dtype=float32),
              array([[2.3900000e+02, 1.7453292e-02]], dtype=float32),
              array([[260.       ,   0.6806784]], dtype=float32),
              array([[363.       ,   0.7330383]], dtype=float32),
              array([[2.4700000e+02, 1.7453292e-02]], dtype=float32),
              array([[306.      ,   0.715585]], dtype=float32),
              array([[585.       ,   0.7853982]], dtype=float32),
              array([[532.       ,   0.7853982]], dtype=float32),
              array([[303.       ,   0.6981317]], dtype=float32),
              array([[2.4100000e+02, 1.7453292e-02]], dtype=float32),
              array([[254.       ,   0.698131

Show image with clustered lines and find vanishing points.

In [11]:
from homography import get_vanishing_point, get_distant_points

In [12]:
vanishing_points = [get_vanishing_point(clusters[c]) for c in clusters]
vanishing_points

[array([-5415.984 , -1955.7689], dtype=float32),
 array([-1668.4064,  8764.042 ], dtype=float32),
 array([-1087.7777,  -159.3853], dtype=float32)]

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    for i, line in enumerate(clusters[c]):
        draw_line(output_image, line, colors[c])
    cv2.circle(output_image, (int(vp[0]), int(vp[1])), 10, colors[c], -1)

show_image("OutputImage", output_image)  

In [13]:
two_points, _ = get_distant_points(vanishing_points)
two_points

[array([-5415.984 , -1955.7689], dtype=float32),
 array([-1668.4064,  8764.042 ], dtype=float32)]

Perform stratification.

In [14]:
vanishing_line = np.cross([*two_points[0], 1], [*two_points[1], 1])
vanishing_line

array([-1.07198109e+04,  3.74757751e+03, -5.07289275e+07])

In [15]:
H1 = np.eye(3)
H1[2] = vanishing_line
H1[2] /= H1[2][2]
print(H1)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 2.11315544e-04 -7.38745662e-05  1.00000000e+00]]


In [16]:
new_image = cv2.warpPerspective(image, H1, (image.shape[1], image.shape[0]))

In [ ]:
show_image("OutputImage", new_image)

# Metric rectification

Get lines from transformed images.

In [17]:
new_lines = get_lines(new_image)
len(new_lines)

38

In [ ]:
output_image = new_image.copy()
for line in new_lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(output_image, line, (r, g, b))
show_image("OutputImage", output_image)

In [18]:
new_clusters = cluster_lines(new_lines, n_clusters=n_clusters)
new_clusters

defaultdict(list,
            {2: [array([[-478.      ,    3.106686]], dtype=float32),
              array([[-277.       ,    3.0194197]], dtype=float32),
              array([[-317.       ,    2.8972466]], dtype=float32),
              array([[-282.       ,    3.0019662]], dtype=float32),
              array([[-261.      ,    3.054326]], dtype=float32),
              array([[-329.     ,    2.86234]], dtype=float32),
              array([[-269.       ,    3.0194197]], dtype=float32),
              array([[-320.       ,    2.8797932]], dtype=float32),
              array([[-281.       ,    3.0194197]], dtype=float32),
              array([[-324.       ,    2.8797932]], dtype=float32),
              array([[-273.       ,    3.0368729]], dtype=float32),
              array([[-265.       ,    3.0368729]], dtype=float32),
              array([[-313.       ,    2.8972466]], dtype=float32),
              array([[-264.       ,    3.0717795]], dtype=float32),
              array([[-315.       ,

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = new_image.copy()
for c in new_clusters:
    for line in new_clusters[c]:
        draw_line(output_image, line, colors[c])

show_image("OutputImage", output_image)  

Find orthogonal lines.

In [19]:
from line import find_orthogonal_lines

In [21]:
orthogonal_lines = find_orthogonal_lines(new_clusters)
orthogonal_lines

[[array([[-32.       ,   2.2340214]], dtype=float32),
  array([[420.       ,   0.6632251]], dtype=float32)],
 [array([[836.       ,   1.3962634]], dtype=float32),
  array([[-282.       ,    3.0019662]], dtype=float32)],
 [array([[537.       ,   0.6806784]], dtype=float32),
  array([[-329.     ,    2.86234]], dtype=float32)]]

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(len(orthogonal_lines))]

output_image = new_image.copy()
for i, lines in enumerate(orthogonal_lines):
    for line in lines:
        draw_line(output_image, line, colors[i])

show_image("OutputImage", output_image)

Make linear equation from 3 orthogonal lines.

In [22]:
from line import get_homogeneous_coordinates

In [37]:
s_true = np.array([1, 1, 0])
best_u, best_s = [], []
best_dist = float('inf')

In [38]:
from scipy.linalg import null_space

In [42]:
for i in range(len(orthogonal_lines)):
    for j in range(i+1, len(orthogonal_lines)):
        A = []
        pair = orthogonal_lines[i]
        l, m = get_homogeneous_coordinates(pair[0]), get_homogeneous_coordinates(pair[1])
        A.append([l[0] * m[0], l[1] * m[1], l[0] * m[1] + l[1] * m[0]])
        
        pair = orthogonal_lines[j]
        l, m = get_homogeneous_coordinates(pair[0]), get_homogeneous_coordinates(pair[1])
        A.append([l[0] * m[0], l[1] * m[1], l[0] * m[1] + l[1] * m[0]])
        
        s = null_space(A)
        
        C_dual = np.zeros((3, 3))
        C_dual[0][0], C_dual[0][1] = s[0], s[1]
        C_dual[1][0], C_dual[1][1] = s[1], s[2]
        C_dual[2][2] = 1
        
        u, s, vh = np.linalg.svd(C_dual)
        dist = np.linalg.norm(s_true - s)
        if dist < best_dist:
            best_dist = dist
            best_u = u
            best_s = s

In [43]:
best_u, best_s

(array([[-0.85241439,  0.        , -0.52286681],
        [-0.52286681,  0.        ,  0.85241439],
        [ 0.        ,  1.        ,  0.        ]]),
 array([1.13816031, 1.        , 0.46126027]))

In [44]:
output_image = cv2.warpPerspective(new_image, best_u, (new_image.shape[1], new_image.shape[0]))

In [45]:
show_image("OutputImage", output_image)